In [52]:
import sys
import os
import re
import csv
import pandas as pd
import subprocess as sp
from pprint import pprint
from pathlib import Path
import yaml

In [53]:
run_dir = Path("/home/roman/dev/gpgpusims/run")
# run_dir = Path("/Users/roman/dev/gpgpusims/run")

In [ ]:
# parse accelsim logs
hw_results = run_dir / "accelsim" / "matrixmul" / "nvidia_geforce_gtx_1080" / "results"
assert hw_results.is_dir()
hw_stat_file = hw_results / "result.txt"
hw_stat_cycles_file = hw_results / "result.cycles.txt"

hw_stat_file_out = hw_results / "result.csv"
hw_stat_cycles_file_out = hw_results / "result.cycles.csv"

In [54]:
def convert_hw_csv(csv_file, output_csv_file):
    with open(csv_file, 'rU') as f:
        reader = csv.reader(f)
        
        # find the start of the csv dump
        prof_pat = re.compile(r"^==\d*==\s*Profiling result:\s*$")
        prof_abort = re.compile(r"^==PROF== Disconnected\s*$")
        for row in reader:
            line = ", ".join(row)
            # print(line)
            if prof_pat.match(line) is not None:
                # print("found start")
                break
            if prof_abort.match(line) is not None:
                # print("found abort")
                break
        
        os.makedirs(output_csv_file.parent, exist_ok=True)
        with open(output_csv_file, 'w') as out:
            output_writer = csv.writer(out)

            # write the valid csv rows to new file
            for row in reader:
                output_writer.writerow(row)

In [55]:
# parse hw stats
hw_results = run_dir / "native" / "matrixmul" / "nvidia_geforce_gtx_1080" / "results"
assert hw_results.is_dir()
hw_stat_file = hw_results / "result.txt"
hw_stat_cycles_file = hw_results / "result.cycles.txt"

hw_stat_file_out = hw_results / "result.csv"
hw_stat_cycles_file_out = hw_results / "result.cycles.csv"

In [56]:
convert_hw_csv(hw_stat_file, hw_stat_file_out)
convert_hw_csv(hw_stat_cycles_file, hw_stat_cycles_file_out)

In [59]:
df = pd.read_csv(hw_stat_file_out)
df

,Start,Duration,Grid X,Grid Y,Grid Z,Block X,Block Y,Block Z,Registers Per Thread,Static SMem,Dynamic SMem,Size,Throughput,SrcMemType,DstMemType,Device,Context,Stream,Name,Correlation_ID
0,us,us,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KB,B,KB,GB/s,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,330085.887000,1.024000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,3.725290,Pinned,Device,NVIDIA GeForce GTX 1080 (0),1.0,14.0,[CUDA memcpy HtoD],123.0
2,330093.919000,1.920000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,1.986821,Pinned,Device,NVIDIA GeForce GTX 1080 (0),1.0,14.0,[CUDA memcpy HtoD],124.0
3,330124.767000,4.864000,1.0,1.0,1.0,32.0,32.0,1.0,30.0,8.000000,0,NaN,NaN,NaN,NaN,NVIDIA GeForce GTX 1080 (0),1.0,14.0,_Z13MatrixMulCUDAILi32EEvPfS0_S0_ii,127.0
4,330152.288000,1.088000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,3.506156,Device,Pinned,NVIDIA GeForce GTX 1080 (0),1.0,14.0,[CUDA memcpy DtoH],132.0


In [58]:
df = pd.read_csv(hw_stat_cycles_file_out)
df

,Device,Context,Stream,Kernel,Correlation_ID,elapsed_cycles_sm,inst_per_warp,branch_efficiency,warp_execution_efficiency,warp_nonpred_execution_efficiency,...,dram_read_transactions,dram_write_transactions,dram_read_throughput,dram_write_throughput,dram_utilization,half_precision_fu_utilization,ecc_transactions,ecc_throughput,dram_read_bytes,dram_write_bytes
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,%,%,%,...,NaN,NaN,MB/s,B/s,NaN,NaN,NaN,B/s,NaN,NaN
1,NVIDIA GeForce GTX 1080 (0),1.0,14.0,_Z13MatrixMulCUDAILi32EEvPfS0_S0_ii,127.0,152810.0,138.0,100.000000,100.000000,98.550725,...,76.0,0.0,740.528715,0.000000,Low (1),Idle (0),0.0,0.000000,2432.0,0.0
